# Tutorial for simulation of CHMP3

In this tutorial we will simulate the CHMP3 protein. Similar to this [paper](https://www.cell.com/structure/fulltext/S0969-2126(10)00395-3). 
The CHMP3 protein contains 2 unstructured domains and 3 rigid domains. The unstructured domain link the rigid domains together and are model with a Gaussian chain.
In the simulation the Gaussian chain is only modeled as an attractive spring potential and no structures are created for it. Explicit linker positions can be generated in post processing.

In the top file we define the 5 domains. The selections here are using the atom ids in the PDB but a [full selection language](https://www.mdanalysis.org/docs/documentation_pages/selections.html) is supported. Note that for the connections the selection has to specify a single atom or `pycomplexes convert` will throw an error.

In [ ]:
%%writefile chmp3.top
box: [200, 200, 200]
topology:
    CHMP3:
        coordinate-file: chmp3.pdb
        domains:
            rigid1:
                type: rigid
                selection: 'bynum 1-132'
            link1:
                type: gaussian
                selection: 'bynum 133-148'
                start_connection: [rigid1, 'bynum 132']
                end_connection: [rigid2, 'bynum 149']
            rigid2:
                type: rigid
                selection: 'bynum 149-164'
            link2:
                type: gaussian
                selection: 'bynum 165-198'
                start_connection: [rigid2, 'bynum 164']
                end_connection: [rigid3, 'bynum 199']
            rigid3:
                type: rigid
                selection: 'bynum 199-214'

With the top file we can use `pycomplexes convert` to generate an cplx file to be used as an input for complexes.

In [ ]:
!pycomplexes convert chmp3.top chmp3.cplx

We also need a configuration file for the simulation

In [ ]:
%%writefile config.yaml
structure: chmp3.cplx
montecarlo:
    algorithm: nvt
    algorithm-params:
        accept-func: metropolis
        temperatur: 300
    seed: 3333
    short-range-cutoff:
        enable: False
output:
    log: complexes.log
    file: chmp3.xtc
    freq: 10
    nstructures: 5
    stat-file: chmp3.stat
    restart-freq: -1

Now we can run the simulation.

In [ ]:
!complexes++ -c config.yaml --nb-threads=1

After the simulation is done we can check on it with vmd. For easier visualization the `pycomplexes visualize` tool can be used that generates a vmd script.

In [ ]:
!pycomplexes visualize -cc config.yaml

The trajectory can now be visualized with `vmd -e chmp3_output.vmd`. The linker beads do not move in the simulation. They are only dummy particles but we can generate new linker positions with the `pycomplexes addlinker` command.

In [ ]:
!pycomplexes addlinker config.yaml chmp3_linker.xtc

To update the VMD script we now have to tell visualize exactly which xtc file we want to use. To better choose between the two simulations we also change the output name of the vmd script. Go ahead and look at the simulation with moving linkers.

In [ ]:
!pycomplexes visualize chmp3.cplx chmp3_reference.pdb chmp3_linker.xtc -o chmp3_linker.vmd --periodic_imgs

The new vmd script can be used to look at the trajectory with explicit linker beads.